In [2]:
import numpy as np

time_bob = np.array([ #[time, empirical; number of observations]
  [10 , 4],
  [20 , 6],
  [30 , 10],
  [40 , 20],
  [50 , 40],
  [60 , 11],
  [70 , 5],
  [80 , 4]
]) 

time_alice = np.array([ #[time, empirical; number of observations]
  [10 , 4],
  [20 , 5],
  [30 , 6],
  [40 , 7],
  [50 , 10],
  [60 , 8],
  [70 , 6],
  [80 , 2]       
])

print(time_bob)
print(time_bob.shape)

[[10  4]
 [20  6]
 [30 10]
 [40 20]
 [50 40]
 [60 11]
 [70  5]
 [80  4]]
(8, 2)


In [3]:
import numpy.random as rd

print("throw a dice 10 times",rd.choice(range(1,7), 10))

print("throw fake dice 30 times",rd.choice(range(1,7), 30, p = [0.4,0.4,0.05,0.05,0.05,0.05]))

print("first column of a table", time_bob[:,0] )
print("second column of a table", time_bob[:,1] )
print("second and third row of a table", time_bob[1:3,:] )
# the rule - in zero based numbering the last element of range is never returned!
print("range(1,4)",list(range(1,4)))

probs = time_bob[:,1] / np.sum(time_bob[:,1])
print("probs",probs)
rd.choice(time_bob[:,0], 20,p=probs )


throw a dice 10 times [3 3 5 6 6 4 2 1 6 3]
throw fake dice 30 times [1 1 1 2 2 2 2 2 1 2 5 2 1 6 2 3 4 1 5 2 2 3 1 2 2 2 1 1 2 6]
first column of a table [10 20 30 40 50 60 70 80]
second column of a table [ 4  6 10 20 40 11  5  4]
second and third row of a table [[20  6]
 [30 10]]
range(1,4) [1, 2, 3]
probs [0.04 0.06 0.1  0.2  0.4  0.11 0.05 0.04]


array([50, 50, 50, 50, 50, 20, 50, 70, 10, 80, 50, 40, 30, 30, 40, 50, 50,
       40, 50, 30])

In [4]:
import numpy.random as rd
rd.seed(0)

rd.choice(time_alice[:,0],100,p=time_alice[:,1]/np.sum(time_alice[:,1]) )

array([50, 60, 50, 50, 40, 50, 40, 70, 80, 40, 60, 50, 50, 70, 10, 20, 10,
       60, 60, 70, 80, 60, 50, 60, 20, 50, 20, 70, 50, 40, 30, 60, 40, 50,
       10, 50, 50, 50, 70, 60, 40, 40, 60, 10, 60, 60, 30, 20, 40, 40, 50,
       40, 80, 20, 30, 20, 50, 30, 50, 30, 20, 20, 50, 20, 30, 40, 60, 20,
       70, 20, 80, 50, 80, 50, 60, 10, 30, 20, 30, 20, 40, 40, 10, 60, 50,
       30, 50, 20, 50, 70, 40, 60, 20, 60, 30, 20, 50, 10, 60, 10])

In [7]:
def min_vals(*args):
    vals = [v for v in args if v is not None]
    if len(vals)==0:
        return None
    return min(vals)

assert min_vals(1,2,3) == 1
assert min_vals(4,2,None) == 2
assert min_vals(None,None,None) == None

In [8]:
# object oriented programming
class Model:    # class (someting kind of "type"/"define data structure")
    x=0  # field x 
    y=0  # field y
    def __init__(self, x, y=100 ):   # constructor
        print("Hey I am constructor I get called whenever an object is created")
        self.x = x
        self.y = y
    def f(self, a):   # method f
        return self.x + a
    
model = Model(999)  # creating an object model of class Model
# an object is a kind of variable

print(model.f(22) )


class Model2(Model): # class inheritance
    # class Model2 is using all of functionality of class Model
    def g(self):
        return self.x - self.y
    
print("working with the second object")
m2 = Model2(88)
print(m2.f(1000)) # inherited method f (from parent)
print(m2.g())  # mrthod g was defined in the child class


Hey I am constructor I get called whenever an object is created
1021
working with the second object
Hey I am constructor I get called whenever an object is created
1088
-12


In [13]:
class ProductionLine:
    alice_action_end = 0  # time measured in seconds
    bob_action_end = None   
    alice_has_box = False #Alice holds or prepares box for Bob
    bob_has_box = False   #Bob works with box

    #storage = 0 # we can model boxes going to storage

    tick  = 0       # simulation step
    clock = 0       # time in simulated world in seconda
    completed_boxes = 0 # completed boxes

    def __init__(self,time_bob,time_alice): #constructor
        self.time_alice = time_alice
        self.time_bob = time_bob        
        self.p_time_alice = time_alice[:,1]/np.sum(time_alice[:,1])
        self.p_time_bob = time_bob[:,1]/np.sum(time_bob[:,1]) 
    
    def step(self):
        #Selecting the next event to happen
        # Calculating the time of the next event to happen
        # Check if this event triggers another conditiona; action such as leaving the queue
        
        #Alice & Bob
        #Unconditional events: Alice completed job, Bob completed job
        #Conditional events: 
        # - Alice starts job (needs to have free hands)
        # - Bob starts job (needs to have free hands and a box must be available)
        
        #unconditional event
        self.clock = min_vals(self.alice_action_end,self.bob_action_end)
        
        # 1st scenario: Alice has just completed the job but Bob has assigned some time, so he is still gluing
        # Alice has comleted assembling the box but she needs to wait for a Bob
        if self.alice_has_box and self.bob_action_end is not None and self.bob_action_end > self.alice_action_end:
            print(self.clock, "Alice needs to wait for Bob ")
            self.clock = self.bob_action_end 
        
        # 2nd scenario: Alice is free and not doing anything --> then she takes a box
        if not self.alice_has_box :
            self.alice_has_box = True
            self.alice_action_end = self.clock + \
                    rd.choice(self.time_alice[:,0],p=self.p_time_alice)
            print(self.clock, "Alice starts doing new box for the time ", self.alice_action_end)

        # 3nd scenario: if Bob has box in his hands and he has just finished manufacturing the box
        if self.bob_has_box and self.bob_action_end <= self.clock:
            print(self.clock, "Bob finishes the box")
            self.completed_boxes += 1
            self.bob_action_end = None
            self.bob_has_box = False

        # if Alice has a box and Alice has finished a job --> Alice is ready to pass the box to Bob
        if self.alice_has_box and (self.alice_action_end is None or self.alice_action_end <= self.clock):
            print(self.clock, "Alice is ready to pass box to Bob ")
            if not self.bob_has_box:            
                self.bob_action_end = self.clock + \
                   rd.choice(self.time_bob[:,0],p=self.p_time_bob)
                print(self.clock, "Bob takes the box from Alice and will be ready at ",self.bob_action_end )
                self.bob_has_box = True
                self.alice_has_box = False
          
        self.tick = self.tick + 1 #increase step size
        
            
    def status(self):
        print("### tick =",self.tick,"clock =",self.clock,"Alice has box", self.alice_has_box,"Alice end =",
              self.alice_action_end,
              "Bob end =",self.bob_action_end, "Bob has box =",self.bob_has_box,
              "completed_boxes =",self.completed_boxes )
        #"storage=",self.storage 
        

In [14]:
rd.seed(10)
# Starting the model
m = ProductionLine(time_bob, time_alice)  # Creating the object with ___init___ constructor
for step in range(40):  
    m.step()
    m.status()

0 Alice starts doing new box for the time  60
### tick = 1 clock = 0 Alice has box True Alice end = 60 Bob end = None Bob has box = False completed_boxes = 0
60 Alice is ready to pass box to Bob 
60 Bob takes the box from Alice and will be ready at  70
### tick = 2 clock = 60 Alice has box False Alice end = 60 Bob end = 70 Bob has box = True completed_boxes = 0
60 Alice starts doing new box for the time  110
### tick = 3 clock = 60 Alice has box True Alice end = 110 Bob end = 70 Bob has box = True completed_boxes = 0
70 Bob finishes the box
### tick = 4 clock = 70 Alice has box True Alice end = 110 Bob end = None Bob has box = False completed_boxes = 1
110 Alice is ready to pass box to Bob 
110 Bob takes the box from Alice and will be ready at  160
### tick = 5 clock = 110 Alice has box False Alice end = 110 Bob end = 160 Bob has box = True completed_boxes = 1
110 Alice starts doing new box for the time  160
### tick = 6 clock = 110 Alice has box True Alice end = 160 Bob end = 160 Bob 

# Homework:

Make plots (for 1 simulation):
### 1) utilization (4 bars)
- of Alice time with and witout Storage (what is percentage of time that Alice is waitng for Bob)
- of Bob's time with and witout Storage (what is percentage of time that Bob is waitng for Alice)
### 2) storage usage over time 

Use either Python or Julia

Provide code + 2 pictures (png format)

- if Alice has storage --> she doesn't wait. So, if there is a storage, than her utilization is 100% time
- Bob can wait because there are no boxes in the storage

utilization if the opposite of how much it's idle (100% of usage = 0% of waiting)

1 of bars is easy for sure:
- when there is a storage - Alice utilization is 100%

**but the question here is rather**: how much time she is actually spending working when there is no storage?